In [ ]:
import numpy as np
import pandas as pd

In [ ]:
%%bash
head -1 /mnt/idms/bdavid/projects/twitter-comb-occ-2017/rg17/rg17_8000_important_en_words_plus_players_results.txt

# Load raw combined occurences

In [ ]:
#file_name = "rg17_20171003_8000_en-players_3h_accurate"
file_name = "rg17_20171012_8000_en-players_3h_accurate"

In [ ]:
#file_path = "/mnt/idms/bdavid/projects/twitter-comb-occ-2017/rg17/rg17_combOcc_results_20170922.txt"
#file_path = "/mnt/idms/bdavid/projects/twitter-comb-occ-2017/rg17/rg17_8000_important_en_words_plus_players_results.txt"
#file_path = "/mnt/idms/bdavid/projects/twitter-comb-occ-2017/rg17/rg17_8000_en-players_6h_accurate.txt"
file_path = "/mnt/idms/bdavid/projects/twitter-comb-occ-2017/rg17/%s.txt" % file_name
#occurences_raw_pd = pd.read_csv(file_path, sep=",", header=None)#, nrows=1000)

In [ ]:
num_cols = 203
raw_rows = []
with open(file_path) as f:
    i = 0
    for line in f:
        i += 1
        splitted = line.rstrip().split(",")
        if len(splitted) == 4:
            # no combined occurences
            continue
        row = list(np.nan for i in range(num_cols))
        row[:len(splitted)] = splitted
        raw_rows.append(tuple(row))
        #if i > 10:
        #    break

In [ ]:
occurences_raw_pd = pd.DataFrame(raw_rows)

In [ ]:
len(occurences_raw_pd)

In [ ]:
occurences_raw_pd.head(5)

# Clean columns

In [ ]:
def clean_cell(x):
    out = x
    try:
        if not x is np.nan:
            for pattern in ["Some(","List(","(",")"," "]:
                if pattern in out:
                    out = out.replace(pattern,"")
    except:
        print(x)
        raise
    return out

In [ ]:
occurences_pd = occurences_raw_pd.applymap(clean_cell)

In [ ]:
num_key_words = 100
kw_text_cols = [0] + [(i+1)*2+1 for i in range(num_key_words)]
kw_count_cols = [(i+1)*2+2 for i in range(num_key_words)]

In [ ]:
kw_text_cols[:4]

### Convert numericals columns

In [ ]:
occurences_pd[kw_count_cols] = occurences_pd[kw_count_cols].astype("float64")

### Introduce new columns

In [ ]:
occurences_pd["key_word"] = occurences_pd[0]
occurences_pd["count"] = occurences_pd[kw_count_cols].max(axis=1)
del occurences_pd[0]

In [ ]:
old_cols = list(occurences_pd.columns)
new_cols = ["start_time","end_time"] + list(range(1,201)) + ["key_word", "count"]
occurences_pd.columns = new_cols
# changing order
occurences_pd = occurences_pd[["start_time","end_time"] + ["key_word", "count"] + list(range(1,201))]

In [ ]:
occurences_pd.head(5)

# Export occurences in tabular format

In [ ]:
output_file = "/mnt/idms/fberes/network/roland_garros/%s.csv" % file_name
print(output_file)
occurences_pd.to_csv(output_file,sep="|",index=False, header=True)